In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Setting plot style
plt.style.use('seaborn')

# 1. Data Loading and Initial Exploration
print("Loading and exploring data...")
data = pd.read_csv('sales_data.csv', encoding='utf-8')

print("Dataset Info:")
print(data.info())
print("\nFirst few rows:")
print(data.head())
print("\nMissing Values:")
print(data.isnull().sum())

# 2. Data Preprocessing
print("\nPreprocessing data...")
def excel_date_to_datetime(excel_date):
    return pd.to_datetime('1899-12-30') + pd.to_timedelta(excel_date, 'D')

# Convert Excel serial date to datetime
data['تاریخ سفارش'] = data['تاریخ سفارش'].apply(excel_date_to_datetime)

# Handle missing values in sales and profit
data['فروش'] = data.apply(lambda row: row['تعداد سفارش'] * row['قیمت واحد'] 
                         if pd.isna(row['فروش']) else row['فروش'], axis=1)
data['سود'] = data.groupby('دسته‌بندی محصول')['سود'].transform(lambda x: x.fillna(x.median()))

# Create profit margin column
data['حاشیه سود'] = (data['سود'] / data['فروش'] * 100).round(2)

print("Data Types After Preprocessing:")
print(data.dtypes)
print("\nMissing Values After Preprocessing:")
print(data.isnull().sum())

# 3. Exploratory Data Analysis
print("\nPerforming exploratory data analysis...")
# Summary statistics
print("Summary Statistics:")
print(data.describe())

# Plot distributions
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 1)
sns.histplot(data['فروش'], bins=50, kde=True)
plt.title('Distribution of Sales')
plt.xlabel('Sales')

plt.subplot(2, 2, 2)
sns.histplot(data['سود'], bins=50, kde=True)
plt.title('Distribution of Profit')
plt.xlabel('Profit')

plt.subplot(2, 2, 3)
sns.histplot(data['حاشیه سود'], bins=50, kde=True)
plt.title('Distribution of Profit Margin')
plt.xlabel('Profit Margin (%)')

plt.subplot(2, 2, 4)
sns.histplot(data['تعداد سفارش'], bins=50, kde=True)
plt.title('Distribution of Order Quantity')
plt.xlabel('Order Quantity')

plt.tight_layout()
plt.savefig('distributions.png')
plt.close()

# Categorical variable analysis
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
data['دسته‌بندی محصول'].value_counts().plot(kind='bar')
plt.title('Product Category Distribution')
plt.xticks(rotation=45)

plt.subplot(1, 3, 2)
data['بخش مشتری'].value_counts().plot(kind='bar')
plt.title('Customer Segment Distribution')
plt.xticks(rotation=45)

plt.subplot(1, 3, 3)
data['روش ارسال'].value_counts().plot(kind='bar')
plt.title('Shipping Method Distribution')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('categorical_distributions.png')
plt.close()

# Correlation matrix
plt.figure(figsize=(10, 8))
numeric_cols = ['تعداد سفارش', 'قیمت واحد', 'فروش', 'سود', 'حاشیه سود']
sns.heatmap(data[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.savefig('correlation_matrix.png')
plt.close()

# 4. Statistical Tests
print("\nRunning statistical tests...")
# Normality test (Shapiro-Wilk)
print("Normality Tests:")
print("Sales:", stats.shapiro(data['فروش'].dropna())[:2])
print("Profit:", stats.shapiro(data['سود'].dropna())[:2])

# ANOVA for profit across customer segments
segments = data['بخش مشتری'].unique()
profit_by_segment = [data[data['بخش مشتری'] == segment]['سود'].dropna() for segment in segments]
anova_result = stats.f_oneway(*profit_by_segment)
print("\nANOVA for Profit across Customer Segments:")
print(f"F-statistic: {anova_result.statistic:.2f}, p-value: {anova_result.pvalue:.4f}")

# ANOVA for sales across product categories
categories = data['دسته‌بندی محصول'].unique()
sales_by_category = [data[data['دسته‌بندی محصول'] == cat]['فروش'].dropna() for cat in categories]
anova_result = stats.f_oneway(*sales_by_category)
print("\nANOVA for Sales across Product Categories:")
print(f"F-statistic: {anova_result.statistic:.2f}, p-value: {anova_result.pvalue:.4f}")

# 5. Advanced Analysis
print("\nPerforming advanced analysis...")
# Time series analysis
monthly_sales = data.groupby(data['تاریخ سفارش'].dt.to_period('M'))['فروش'].sum()
monthly_profit = data.groupby(data['تاریخ سفارش'].dt.to_period('M'))['سود'].sum()

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
monthly_sales.plot()
plt.title('Monthly Sales Trend')
plt.xticks(rotation=45)
plt.xlabel('Month')
plt.ylabel('Sales')

plt.subplot(1, 2, 2)
monthly_profit.plot()
plt.title('Monthly Profit Trend')
plt.xticks(rotation=45)
plt.xlabel('Month')
plt.ylabel('Profit')

plt.tight_layout()
plt.savefig('time_series.png')
plt.close()

# Top 10 customers by sales
top_customers = data.groupby('نام مشتری')['فروش'].sum().nlargest(10)
plt.figure(figsize=(10, 6))
top_customers.plot(kind='bar')
plt.title('Top 10 Customers by Sales')
plt.xticks(rotation=45)
plt.ylabel('Total Sales')
plt.savefig('top_customers.png')
plt.close()

# Profitability by category and segment
pivot_profit = data.pivot_table(values='حاشیه سود', 
                              index='دسته‌بندی محصول', 
                              columns='بخش مشتری', 
                              aggfunc='mean')
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_profit, annot=True, fmt='.2f', cmap='RdYlGn')
plt.title('Average Profit Margin by Category and Segment (%)')
plt.savefig('profitability_heatmap.png')
plt.close()

# 6. Key Insights and Recommendations
print("\nKey Insights and Recommendations:")
print("1. Sales Distribution: The sales distribution is right-skewed, indicating many small orders and few large orders.")
print("2. Profit Variability: Profit shows significant variability, with some categories (e.g., Technology) having higher margins.")
print("3. Customer Segments: The ANOVA test suggests differences in profit across customer segments (p-value < 0.05).")
print("4. Trends: Monthly sales show seasonal patterns that could be leveraged for inventory planning.")
print("5. Top Customers: A small number of customers contribute significantly to sales.")
print("\nRecommendations:")
print("- Focus on high-margin product categories like Technology for promotional campaigns.")
print("- Investigate negative profit orders to identify pricing or cost issues.")
print("- Develop targeted marketing for top customers to maintain loyalty.")
print("- Optimize shipping methods based on profitability analysis.")

In [ ]:
for cat in list(set(data_set['product_category'])):
    filtered_data = data_set[data_set['product_category'] == cat]['profit/sale']

# Create distribution plot
    plt.figure(figsize=(10, 6))
    sns.histplot(filtered_data, bins=30, kde=True, color='#1f77b4')
    plt.title(f'Distribution of Sales for {cat}')
    plt.xlabel('Sales')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()
    print(f"Summary Statistics for Sales in {cat}:")
    print(filtered_data.describe())
    
    data_set = pd.get_dummies(data_set, columns=['product_category', 'customer_division', 'shiping_method'], prefix=['cat', 'div', 'ship'])
data_set.head()

In [ ]:
groups = []
for col in df.filter(like='category_'):  # Adjust to match your one-hot columns
    groups.append(df[df[col] == 1]['target'])

# 2. Perform one-way ANOVA
f_stat, p_value = stats.f_oneway(*groups)
print(f"ANOVA Results: F-statistic={f_stat:.3f}, p-value={p_value:.4f}")

# 3. Post-hoc test (if ANOVA is significant)
if p_value < 0.05:  # Common significance threshold
    print("\nSignificant difference found. Running post-hoc analysis...")
    # Combine all data for Tukey test
    melted_df = df.melt(id_vars=['target'], 
                        value_vars=df.filter(like='category_').columns,
                        var_name='category',
                        value_name='is_member')
    tukey = pairwise_tukeyhsd(endog=melted_df['target'],
                             groups=melted_df[melted_df['is_member'] == 1]['category'],
                             alpha=0.05)
    print(tukey)
else:
    print("\nNo significant difference between groups")

In [ ]:
# %% [markdown]
# # Sales Data Analysis with Machine Learning
# 
# This notebook analyzes the sales dataset using machine learning techniques to predict profits and identify key patterns.

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# %%
# Load the data
df = pd.read_csv('sales_dataset_en_headers.csv')

# %% [markdown]
# ## 1. Data Exploration and Cleaning

# %%
# Display basic info
print("Data shape:", df.shape)
print("\nFirst 5 rows:")
display(df.head())

# %%
# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# %%
# Clean the data
# Remove rows with missing values in key columns
df = df.dropna(subset=['sale', 'profit'])

# Clean currency columns
def clean_currency(x):
    if isinstance(x, str):
        x = x.replace('$', '').replace(',', '').replace('(', '-').replace(')', '')
    return float(x)

df['sale'] = df['sale'].apply(clean_currency)
df['profit'] = df['profit'].apply(clean_currency)

# Convert order_date to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Extract year and month
df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month

# %%
# Basic statistics
print("\nBasic statistics:")
display(df.describe())

# %% [markdown]
# ## 2. Exploratory Data Analysis (EDA)

# %%
# Plot sales and profit over time
plt.figure(figsize=(12, 6))
df.groupby('year')['sale'].sum().plot(kind='bar')
plt.title('Total Sales by Year')
plt.ylabel('Total Sales ($)')
plt.show()

# %%
# Profit by product category
plt.figure(figsize=(10, 6))
sns.boxplot(x='product_category', y='profit', data=df)
plt.title('Profit Distribution by Product Category')
plt.xticks(rotation=45)
plt.show()

# %%
# Correlation heatmap
plt.figure(figsize=(8, 6))
numeric_cols = ['order_count', 'unit_price', 'sale', 'profit', 'year', 'month']
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# %% [markdown]
# ## 3. Feature Engineering

# %%
# Encode categorical variables
label_encoders = {}
categorical_cols = ['shiping_method', 'customer_division', 'product_category']

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Create total_price feature
df['total_price'] = df['order_count'] * df['unit_price']

# %%
# Prepare features and target
X = df[['order_count', 'unit_price', 'shiping_method', 
        'customer_division', 'product_category', 'year', 'month', 'total_price']]
y = df['profit']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# %% [markdown]
# ## 4. Machine Learning Models

# %% [markdown]
# ### 4.1 Profit Prediction with Random Forest

# %%
# Train Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)

# Predictions
y_pred = rf.predict(X_test_scaled)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Random Forest Performance:")
print(f"MSE: {mse:.2f}")
print(f"R2 Score: {r2:.2f}")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nFeature Importance:")
display(feature_importance)

# %% [markdown]
# ### 4.2 Customer Segmentation with K-Means Clustering

# %%
# Prepare data for clustering
cluster_features = df.groupby('customer_name').agg({
    'sale': 'sum',
    'profit': 'mean',
    'order_count': 'count'
}).reset_index()

# Scale features
cluster_data = cluster_features[['sale', 'profit', 'order_count']]
cluster_data_scaled = scaler.fit_transform(cluster_data)

# Determine optimal clusters using elbow method
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(cluster_data_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

# %%
# Apply K-Means with selected K
k = 3  # Based on elbow method
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(cluster_data_scaled)

# Add clusters to data
cluster_features['cluster'] = clusters

# Visualize clusters with PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(cluster_data_scaled)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=clusters, palette='viridis')
plt.title('Customer Segmentation with K-Means (PCA Reduced)')
plt.show()

# Analyze clusters
print("\nCluster Characteristics:")
display(cluster_features.groupby('cluster').mean())

# %% [markdown]
# ## 5. Time Series Analysis (Optional)

# %%
# Monthly sales trend
monthly_sales = df.groupby(['year', 'month'])['sale'].sum().reset_index()
plt.figure(figsize=(12, 6))
sns.lineplot(x='month', y='sale', hue='year', data=monthly_sales, marker='o')
plt.title('Monthly Sales Trend by Year')
plt.show()

# %% [markdown]
# ## 6. Recommendations and Conclusion

# %%
# Based on our analysis:
print("Key Insights:")
print("1. The most important features for profit prediction are total_price, unit_price, and order_count.")
print("2. Customers can be segmented into 3 distinct groups based on their purchasing behavior.")
print("3. There are clear seasonal patterns in sales data that could inform inventory planning.")
print("\nRecommendations:")
print("- Focus on high-margin products identified in the product category analysis")
print("- Tailor marketing strategies to different customer segments")
print("- Optimize inventory based on seasonal sales patterns")